### Synaesthesia ###

Time series data of parcelations of 127 subjects, both synaesthetic and control

In [1]:
# Imports

import pandas as pd
import numpy as np
import scipy.io
import pickle 
import json
import torch

## 4. Model

### 4.1 Vanilla GCN


In [2]:
# https://colab.research.google.com/drive/1I8a0DfQ3fI7Njc62__mVXUlcAleUclnb?usp=sharing


# Install required packages.
import os
import torch
os.environ['TORCH'] = torch.__version__
print(torch.__version__)

# %pip install -q torch-scatter -f https://data.pyg.org/whl/torch-${TORCH}.html
# %pip install -q torch-sparse -f https://data.pyg.org/whl/torch-${TORCH}.html
# %pip install -q git+https://github.com/pyg-team/pytorch_geometric.git

# or use the provided conda environment


2.4.1


In [3]:
# Load Pickled data

file = open('./data/data_2.pkl', 'rb')   
pickled_data = pickle.load(file)
file.close()

dataset = pickled_data
print()
print(f'Dataset: {dataset}:')
print('====================')
print(f'Number of graphs: {len(dataset)}')
# print(f'Number of features: {dataset.num_features}')
# print(f'Number of classes: {dataset.num_classes}')

first_datum = dataset[0]  # Get the first graph object.

print()
print(first_datum)
print('=============================================================')

# Gather some statistics about the first graph.
print(f'Number of nodes: {first_datum.num_nodes}')
print(f'Number of edges: {first_datum.num_edges}')
print(f'Average node degree: {first_datum.num_edges / first_datum.num_nodes}')
print(f'Has isolated nodes: {first_datum.has_isolated_nodes()}')
print(f'Has self-loops: {first_datum.has_self_loops()}')
print(f'Is undirected: {first_datum.is_undirected()}')


Dataset: [Data(x=[360, 360], edge_index=[2, 25916], edge_attr=[25916], y=1, pos=[360, 360]), Data(x=[360, 360], edge_index=[2, 25916], edge_attr=[25916], y=1, pos=[360, 360]), Data(x=[360, 360], edge_index=[2, 25916], edge_attr=[25916], y=1, pos=[360, 360]), Data(x=[360, 360], edge_index=[2, 25916], edge_attr=[25916], y=1, pos=[360, 360]), Data(x=[360, 360], edge_index=[2, 25917], edge_attr=[25917], y=1, pos=[360, 360]), Data(x=[360, 360], edge_index=[2, 25917], edge_attr=[25917], y=1, pos=[360, 360]), Data(x=[360, 360], edge_index=[2, 25916], edge_attr=[25916], y=0, pos=[360, 360]), Data(x=[360, 360], edge_index=[2, 25916], edge_attr=[25916], y=1, pos=[360, 360]), Data(x=[360, 360], edge_index=[2, 25916], edge_attr=[25916], y=0, pos=[360, 360]), Data(x=[360, 360], edge_index=[2, 25916], edge_attr=[25916], y=1, pos=[360, 360]), Data(x=[360, 360], edge_index=[2, 25916], edge_attr=[25916], y=1, pos=[360, 360]), Data(x=[360, 360], edge_index=[2, 25916], edge_attr=[25916], y=1, pos=[360, 

In [4]:
# torch.manual_seed(12345)

train_dataset = dataset[:100]
test_dataset = dataset[100:]

print(f'Number of training graphs: {len(train_dataset)}')
print(f'Number of test graphs: {len(test_dataset)}')

Number of training graphs: 100
Number of test graphs: 27


In [5]:
from torch_geometric.loader import DataLoader

train_loader = DataLoader(train_dataset, batch_size=10, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=10, shuffle=False)

for step, data in enumerate(train_loader):
    print(f'Step {step + 1}:')
    print('=======')
    print(f'Number of graphs in the current batch: {data.num_graphs}')
    print(data)
    print()

Step 1:
Number of graphs in the current batch: 10
DataBatch(x=[3600, 360], edge_index=[2, 259163], edge_attr=[259163], y=[10], pos=[3600, 360], batch=[3600], ptr=[11])

Step 2:
Number of graphs in the current batch: 10
DataBatch(x=[3600, 360], edge_index=[2, 259162], edge_attr=[259162], y=[10], pos=[3600, 360], batch=[3600], ptr=[11])

Step 3:
Number of graphs in the current batch: 10
DataBatch(x=[3600, 360], edge_index=[2, 259163], edge_attr=[259163], y=[10], pos=[3600, 360], batch=[3600], ptr=[11])

Step 4:
Number of graphs in the current batch: 10
DataBatch(x=[3600, 360], edge_index=[2, 259161], edge_attr=[259161], y=[10], pos=[3600, 360], batch=[3600], ptr=[11])

Step 5:
Number of graphs in the current batch: 10
DataBatch(x=[3600, 360], edge_index=[2, 259162], edge_attr=[259162], y=[10], pos=[3600, 360], batch=[3600], ptr=[11])

Step 6:
Number of graphs in the current batch: 10
DataBatch(x=[3600, 360], edge_index=[2, 259161], edge_attr=[259161], y=[10], pos=[3600, 360], batch=[3600

In [6]:
from torch.nn import Linear
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.nn import global_mean_pool


class GCN(torch.nn.Module):
    def __init__(self, hidden_channels):
        super(GCN, self).__init__()
        torch.manual_seed(12345)
        self.conv1 = GCNConv(360, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, hidden_channels)
        self.conv3 = GCNConv(hidden_channels, hidden_channels)
        self.lin = Linear(hidden_channels, 2)

    def forward(self, x, edge_index, batch):
        # 1. Obtain node embeddings 
        x = self.conv1(x, edge_index)
        x = x.relu()
        x = self.conv2(x, edge_index)
        x = x.relu()
        x = self.conv3(x, edge_index)

        # 2. Readout layer
        x = global_mean_pool(x, batch)  # [batch_size, hidden_channels]

        # 3. Apply a final classifier
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.lin(x)
        
        return x

model = GCN(hidden_channels=64)
print(model)


def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)




print(count_parameters(model))

GCN(
  (conv1): GCNConv(360, 64)
  (conv2): GCNConv(64, 64)
  (conv3): GCNConv(64, 64)
  (lin): Linear(in_features=64, out_features=2, bias=True)
)
31554


In [7]:
model = GCN(hidden_channels=64)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.CrossEntropyLoss()


def train():
    model.train()

    for data in train_loader:  # Iterate in batches over the training dataset.
         out = model(data.x, data.edge_index, data.batch)  # Perform a single forward pass.
         loss = criterion(out, data.y)  # Compute the loss.
         loss.backward()  # Derive gradients.
         optimizer.step()  # Update parameters based on gradients.
         optimizer.zero_grad()  # Clear gradients.

def test(loader):
    model.eval()

    true_p = 0
    true_n = 0
    false_p = 0
    false_n = 0

    correct = 0
    for data in loader:  # Iterate in batches over the training/test dataset.
        out = model(data.x, data.edge_index, data.batch)  
        loss = criterion(out, data.y)  # Compute the loss.
        pred = out.argmax(dim=1)  # Use the class with highest probability.
        correct += int((pred == data.y).sum())  # Check against ground-truth labels.
        accuracy = (correct / len(loader.dataset))

        for n in range(len(data.y)):
            if pred[n]==0 and data.y[n]==0 :
                true_n += 1
            elif pred[n]==0 and data.y[n]==1 :
                false_n += 1
            elif pred[n]==1 and data.y[n]==0 :
                false_p += 1
            elif pred[n]==1 and data.y[n]==1 :
                true_p += 1

        precision = true_p / (true_p + false_p)
        recall = true_p / (true_p + false_n)
        f_one = 2 * precision * recall / (precision + recall)

        
    return accuracy, loss, f_one  # Derive ratio of correct predictions.


for epoch in range(1, 201):
    train()
    train_acc, train_loss, train_f_one = test(train_loader)
    test_acc, test_loss, test_f_one = test(test_loader)
    print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}, Train Loss: {train_loss:.4f}, Train F1 Score: {train_f_one:.4f}, Test Acc: {test_acc:.4f},  Test Loss: {test_loss:.4f}, Test F1 Score: {test_f_one:.4f}')





Epoch: 001, Train Acc: 0.7900, Train Loss: 0.8529, Train F1 Score: 0.8827, Test Acc: 0.8519,  Test Loss: 0.1551, Test F1 Score: 0.9200
Epoch: 002, Train Acc: 0.7900, Train Loss: 0.5416, Train F1 Score: 0.8827, Test Acc: 0.8519,  Test Loss: 0.1173, Test F1 Score: 0.9200
Epoch: 003, Train Acc: 0.7900, Train Loss: 0.5084, Train F1 Score: 0.8827, Test Acc: 0.8519,  Test Loss: 0.3346, Test F1 Score: 0.9200
Epoch: 004, Train Acc: 0.7900, Train Loss: 0.3906, Train F1 Score: 0.8827, Test Acc: 0.8519,  Test Loss: 0.1870, Test F1 Score: 0.9200
Epoch: 005, Train Acc: 0.7900, Train Loss: 0.5856, Train F1 Score: 0.8827, Test Acc: 0.8519,  Test Loss: 0.4000, Test F1 Score: 0.9200
Epoch: 006, Train Acc: 0.7900, Train Loss: 0.6067, Train F1 Score: 0.8827, Test Acc: 0.8519,  Test Loss: 0.3405, Test F1 Score: 0.9200
Epoch: 007, Train Acc: 0.7900, Train Loss: 0.2993, Train F1 Score: 0.8827, Test Acc: 0.8519,  Test Loss: 0.1581, Test F1 Score: 0.9200
Epoch: 008, Train Acc: 0.7900, Train Loss: 0.6666, Trai

### 4.2 GraphConv

In [8]:
from torch_geometric.nn import GraphConv


class GNN(torch.nn.Module):
    def __init__(self, hidden_channels):
        super(GNN, self).__init__()
        torch.manual_seed(12345)
        self.conv1 = GraphConv(360, hidden_channels)
        self.conv2 = GraphConv(hidden_channels, hidden_channels)
        self.conv3 = GraphConv(hidden_channels, hidden_channels)
        self.lin = Linear(hidden_channels, 2)

    def forward(self, x, edge_index, batch):
        x = self.conv1(x, edge_index)
        x = x.relu()
        x = self.conv2(x, edge_index)
        x = x.relu()
        x = self.conv3(x, edge_index)

        x = global_mean_pool(x, batch)

        x = F.dropout(x, p=0.5, training=self.training)
        x = self.lin(x)
        
        return x

model = GNN(hidden_channels=64)
print(model)
print(count_parameters(model))

GNN(
  (conv1): GraphConv(360, 64)
  (conv2): GraphConv(64, 64)
  (conv3): GraphConv(64, 64)
  (lin): Linear(in_features=64, out_features=2, bias=True)
)
62786


In [9]:

model = GNN(hidden_channels=64)
print(model)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

for epoch in range(1, 201):
    train()
    train_acc, train_loss, train_f_one = test(train_loader)
    test_acc, test_loss, test_f_one = test(test_loader)
    print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}, Train Loss: {train_loss:.4f}, Train F1 Score: {train_f_one:.4f}, Test Acc: {test_acc:.4f},  Test Loss: {test_loss:.4f}, Test F1 Score: {test_f_one:.4f}')

GNN(
  (conv1): GraphConv(360, 64)
  (conv2): GraphConv(64, 64)
  (conv3): GraphConv(64, 64)
  (lin): Linear(in_features=64, out_features=2, bias=True)
)
Epoch: 001, Train Acc: 0.7900, Train Loss: 46084.3125, Train F1 Score: 0.8827, Test Acc: 0.8519,  Test Loss: 0.0000, Test F1 Score: 0.9200
Epoch: 002, Train Acc: 0.7200, Train Loss: 2025.0199, Train F1 Score: 0.8313, Test Acc: 0.7407,  Test Loss: 172.2962, Test F1 Score: 0.8511
Epoch: 003, Train Acc: 0.7600, Train Loss: 1446.6771, Train F1 Score: 0.8554, Test Acc: 0.6296,  Test Loss: 1156.7422, Test F1 Score: 0.7619
Epoch: 004, Train Acc: 0.6300, Train Loss: 583.6476, Train F1 Score: 0.7582, Test Acc: 0.4815,  Test Loss: 1634.7032, Test F1 Score: 0.6111
Epoch: 005, Train Acc: 0.8000, Train Loss: 0.6192, Train F1 Score: 0.8876, Test Acc: 0.8519,  Test Loss: 0.5635, Test F1 Score: 0.9200
Epoch: 006, Train Acc: 0.8000, Train Loss: 0.6225, Train F1 Score: 0.8876, Test Acc: 0.8519,  Test Loss: 0.1998, Test F1 Score: 0.9200
Epoch: 007, Trai

### 4.3 BrainGNN

In [17]:
# https://github.com/xxlya/BrainGNN_Pytorch

%run .//BrainGNN_Pytorch//BrainGNN_main.py --data_pickle="./data/data_2.pkl" --nclass=2 --fold=1


97
15
15
model: Network(
  (n1): Sequential(
    (0): Linear(in_features=360, out_features=8, bias=False)
    (1): ReLU()
    (2): Linear(in_features=8, out_features=11520, bias=True)
  )
  (conv1): MyNNConv(360, 32)
  (pool1): TopKPooling(32, ratio=0.5, multiplier=1)
  (n2): Sequential(
    (0): Linear(in_features=360, out_features=8, bias=False)
    (1): ReLU()
    (2): Linear(in_features=8, out_features=1024, bias=True)
  )
  (conv2): MyNNConv(32, 32)
  (pool2): TopKPooling(32, ratio=0.5, multiplier=1)
  (fc1): Linear(in_features=128, out_features=32, bias=True)
  (bn1): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc2): Linear(in_features=32, out_features=512, bias=True)
  (bn2): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc3): Linear(in_features=512, out_features=2, bias=True)
)
train...........
LR 0.01
*====**
0m 1s
Epoch: 000, Train Acc: 0.7732, Train NLLLoss: 0.0079, Train Overall Loss: 133.3611, Tra